In [536]:
# import needed library
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math

In [537]:
#load iris dataset
data_iris = load_iris()
iris_X, iris_y = load_iris(return_X_y=True)
feature_iris = data_iris['feature_names']

#load play tennis dataset
play_tennis =  pd.read_csv('play_tennis.csv')
play_tennis = play_tennis.drop('day',axis=1)

In [538]:
#transform iris into dataframe
iris_X=pd.DataFrame(iris_X)
iris_y=pd.DataFrame(iris_y)

In [539]:
#create index so be merge
iris_X=iris_X.reset_index()
iris_y=iris_y.reset_index()

In [540]:
iris_y.rename(columns = {0:4}, inplace = True) 

In [541]:
#merge dataset iris
iris=iris_X.merge(iris_y)

In [542]:
#drop index
iris.drop("index",axis=1,inplace=True)

In [543]:
iris.rename(columns = {0:feature_iris[0],1:feature_iris[1],2:feature_iris[2],3:feature_iris[3],4:"target"}, inplace = True)

In [544]:
def entropy(parsed_data, target_attribute):
    parsed_value_target = {}
    total_value_target = 0
  
    for i in parsed_data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

            total_value_target += 1
  
    log_result = 0

    for i in parsed_value_target:
        log_result += float(parsed_value_target[i])/total_value_target * math.log((float(parsed_value_target[i])/total_value_target), 2)
  
    return -1 * log_result

In [545]:
# hasn't handle after universal entropy
def information_gain(data, gain_attribute, target_attribute):
    gain_result = 0
    attribute_entropy_result = 0
    parsed_attribute_count = {}
    total_attribute_count = 0
    
    for i in data[gain_attribute]:
        if i is not None:
            if i not in parsed_attribute_count:
                parsed_attribute_count[i] = 1
            else:
                parsed_attribute_count[i] += 1
            
            total_attribute_count += 1
    
    for i in parsed_attribute_count:
        parsed_data = data.loc[data[gain_attribute]==i]
        attribute_entropy_result += float(parsed_attribute_count[i])/total_attribute_count * entropy(parsed_data, target_attribute)    

    gain_result += entropy(data,target_attribute) + (-1 * attribute_entropy_result)
    return gain_result

In [546]:
def best_attribute(data,target_attribute):
    gain_attribute = {
        'value': 0,
        'name': ''
    }
    
    
    for i in data.columns:
        if (i != target_attribute):
            if information_gain(data, i, target_attribute) > gain_attribute['value']:
                gain_attribute['value'] = information_gain(data, i, target_attribute)
                gain_attribute['name'] = i

    return gain_attribute['name']

In [547]:
entropy(play_tennis,"play")

0.9402859586706309

In [548]:
information_gain(play_tennis, "outlook", "play")

0.2467498197744391

In [549]:
best_attribute(play_tennis,'play')

'outlook'

In [550]:
import math

class Node:
    def __init__(self, attribute=None, label=None, vertex=None):
        self.attribute = attribute
        self.label = label
        self.vertex = vertex
        self.children = {}
        self.most_common_label = None
    
    def set_most_common_label(self, most_common_label):
        self.most_common_label = most_common_label
        
    def get_most_common_label(self):
        return self.most_common_label
        
    def setAttribute(self, attribute):
        self.attribute = attribute

    def setLabel(self, label):
        self.label = label
        
    def setVertex(self, vertex):
        self.vertex = vertex
  
    def addChildren(self, attributeValue, node):
        self.children[attributeValue] = node
    
    def getChildren(self):
        return self.children
    
    def getLabel(self):
        return self.label
    
    def getVertex(self):
        return self.vertex

In [551]:
def get_most_common_label(data, target_attribute):
    parsed_value_target = {}
  
    for i in data[target_attribute]:
        if i is not None:
            if i not in parsed_value_target:
                parsed_value_target[i] = 1
            else:
                parsed_value_target[i] += 1

    most_common = {
        'value': 0,
        'name': ''
    }
    
    for i in parsed_value_target:
        if parsed_value_target[i] > most_common['value']:
            most_common['value'] = parsed_value_target[i]
            most_common['name'] = i
    
    return most_common['name']

In [552]:
play_tennis['play'].unique()[0]

'No'

In [553]:
play_tennis['play'].unique()

array(['No', 'Yes'], dtype=object)

In [554]:
def id3(data, target_attribute, vertex_=None):
    node = Node(vertex = vertex_)
    if data[target_attribute].nunique()==1:
        node.setLabel(data[target_attribute].unique()[0])
        return node
     
    elif len(play_tennis.columns)==1:
        node.setLabel(get_most_common_label(data, target_attribute))
        return node
    
    else:
        best_attribute_ = best_attribute(data,target_attribute)
        node.setAttribute(best_attribute_)
        for i in data[best_attribute_].unique():
            node.addChildren(i,id3(data.loc[data[best_attribute_]==i],target_attribute, i))
            most_comm = None
            occ = 0;
            for i in data[target_attribute].unique():
                if data[data[target_attribute] == i].shape[0] >  occ:
                    most_comm = i
                    occ = data[data[target_attribute] == i].shape[0]
            node.set_most_common_label(most_comm)        
            
    return node

In [555]:
play_tennis['play'][0]

'No'

In [556]:
id3(play_tennis,"play")

In [557]:
def print_tree(node,depth):
    if node.label is not None: 
        print("    "*(depth+1) +node.label)
    else:
        print("    "*depth + "["+ node.attribute +"]")
        for i in node.children:
            print("----"*(depth+1) +i)
            print_tree(node.children[i],depth+1)        
        

In [558]:
def copy_tree(node):
    temp_node = Node()
    if node.label is not None: 
        temp_node.setLabel(node.label)
    else:
        temp_node.setAttribute(node.attribute)
        
        
    
        
        for i in node.children:
            temp= Node()
            temp= node.children[i]
            temp_node.addChildren(i, temp)
    return temp_node

In [559]:
print_tree(id3(play_tennis, "play"),0)

[outlook]
----Sunny
    [humidity]
--------High
            No
--------Normal
            Yes
----Overcast
        Yes
----Rain
    [wind]
--------Weak
            Yes
--------Strong
            No


In [560]:
print(id3(play_tennis, "play").children['Sunny'].children['High'].label)

No


In [561]:
test=play_tennis[0:3]

In [562]:
test_X = test.drop('play',axis=1)

In [563]:
def check_tree(node,data,index,result, target_attribute):
    if node.label is not None: 
        result.append(node.getLabel())
    else:
        if data.loc[index, node.attribute] is None:
            result.append(node.get_most_common_label())
        for i in node.children:
            if i==data.loc[index,node.attribute]:
                check_tree(node.children[i],data,index,result, target_attribute)

In [564]:
def pred(data,model,target_attribute):
    result = []
    for i in range(len(data)):
        check_tree(model,data[i:i+1],i,result, target_attribute)
    return pd.DataFrame(result)
    

In [565]:
test_X['humidity'] = None

In [566]:
test_X

,outlook,temp,humidity,wind
0,Sunny,Hot,None,Weak
1,Sunny,Hot,None,Strong
2,Overcast,Hot,None,Weak


In [567]:
model=id3(play_tennis, "play")
res = pred(test_X,model, "play")

In [568]:
res

,0
0,No
1,No
2,Yes


In [569]:
def accuracy(pred,data):
    cnt = 0
    for i in range(len(pred)):
        if pred.loc[i] == data.loc[i]:
            cnt+=1
    return cnt*100/len(pred)

In [570]:
# accuracy(play_tennis['play'],x,'play')

In [571]:
play_tennis.loc[[0]]

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No


In [572]:
play_tennis.shape[0]

14

In [573]:
play_tennis_column = play_tennis.columns
play_tennis_column

Index(['outlook', 'temp', 'humidity', 'wind', 'play'], dtype='object')

In [574]:
data_temp = pd.DataFrame(columns=play_tennis_column)
data_temp

,outlook,temp,humidity,wind,play


In [575]:
def get_data_validate(data):
    data_column = data.columns
    data_20 = pd.DataFrame(columns=data_column)
    data_80 = pd.DataFrame(columns=data_column)
    print(data_column)
    count_20 = 0
    count_80 = 0
    
    for i in range(data.shape[0]):
        if(i%5 == 0):
            # Pass the row elements as key value pairs to append() function 
            data_20 = data_20.append(data.loc[[i]] , ignore_index=True)
        else:
            data_80 = data_80.append(data.loc[[i]] , ignore_index=True)
    return data_20, data_80

In [576]:
data20, data80 = get_data_validate(play_tennis)
data20

Index(['outlook', 'temp', 'humidity', 'wind', 'play'], dtype='object')


,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Rain,Cool,Normal,Strong,No
2,Sunny,Mild,Normal,Strong,Yes


In [577]:
data20, data80 = get_data_validate(play_tennis)
data80

Index(['outlook', 'temp', 'humidity', 'wind', 'play'], dtype='object')


,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Strong,No
1,Overcast,Hot,High,Weak,Yes
2,Rain,Mild,High,Weak,Yes
3,Rain,Cool,Normal,Weak,Yes
4,Overcast,Cool,Normal,Strong,Yes
5,Sunny,Mild,High,Weak,No
6,Sunny,Cool,Normal,Weak,Yes
7,Rain,Mild,Normal,Weak,Yes
8,Overcast,Mild,High,Strong,Yes
9,Overcast,Hot,Normal,Weak,Yes


In [578]:
def prune_tree(node, attribute, vertex):
    if(node.label is not None):
        return node,0
    elif(node.attribute == attribute and node.vertex == vertex):
        node.children = {}
        node.attribute = None
        node.label = node.most_common_label
        
        return node,1
    else:
        for i in node.children:
            a,b = prune_tree(node.children[i], attribute, vertex)
            
        return node,b

In [579]:
print_tree(id3(play_tennis, "play"),0)
# print_tree(prune_tree(id3(play_tennis, "play"),"humidity"), 0)

[outlook]
----Sunny
    [humidity]
--------High
            No
--------Normal
            Yes
----Overcast
        Yes
----Rain
    [wind]
--------Weak
            Yes
--------Strong
            No


In [596]:
def post_pruning(data, node, current_node, children_node, target_attribute):
    if children_node == 0:
        return node
    else:
        for i in current_node.children:
            if current_node.vertex is None: 
                #print(i,1)
                post_pruning(data, node, current_node.children[i], children_node-1, target_attribute)
            else:
                if current_node.attribute is not None:
                    data20, data80 = get_data_validate(data)
                    #print(current_node.attribute,2)
                    save = copy_tree(current_node)
                    temp_node,c = prune_tree(node, current_node.attribute, current_node.vertex)
                    train=data20.drop(target_attribute,axis=1)
                    #print_tree(node,0)
                    temp_node_pred = pred(train, node, target_attribute)
                    temp_node_accuracy = accuracy(temp_node_pred[0], data20[target_attribute])
                    #print(temp_node_pred[0],data20)
                    #print(temp_node_accuracy)
                    if temp_node_accuracy == 100:
                        post_pruning(data, temp_node, current_node, children_node, target_attribute)
                    else :
                        
                        node.addChildren(current_node.vertex, save)
                        #print_tree(node,0)


                        post_pruning(data, node, save.children[i], children_node, target_attribute)
        return node

In [597]:
node_play_tennis = id3(play_tennis, "play")

In [598]:
node_play_tennis = post_pruning(play_tennis, node_play_tennis,node_play_tennis, len(node_play_tennis.children)+1, 'play')

Index(['outlook', 'temp', 'humidity', 'wind', 'play'], dtype='object')
Index(['outlook', 'temp', 'humidity', 'wind', 'play'], dtype='object')


In [599]:
print_tree(node_play_tennis, 0)

[outlook]
----Sunny
    [humidity]
--------High
            No
--------Normal
            Yes
----Overcast
        Yes
----Rain
    [wind]
--------Weak
            Yes
--------Strong
            No


In [600]:
data20

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Rain,Cool,Normal,Strong,No
2,Sunny,Mild,Normal,Strong,Yes
